<a href="https://colab.research.google.com/github/bbang3/korean-text-augmentation/blob/klue/bert_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/korean-text-augmentation

/content/drive/MyDrive/korean-text-augmentation


# Import

In [4]:
!pip install transformers[pytorch]
!pip install datasets
!pip install accelerate
!pip install evaluate
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


In [5]:
import os

import transformers
import torch
import pandas as pd
from tqdm.auto import tqdm

from torch.utils.data import DataLoader

from transformers import BertModel, BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForMaskedLM
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling

import wandb

In [6]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [48]:
os.environ["WANDB_PROJECT"] = "BERT MLM Training"

In [9]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [74]:
import gc
gc.collect()
torch.cuda.empty_cache()
del model

In [13]:
train_path = 'data/train_low.csv'
val_path = 'data/val_low.csv'
dataset = load_dataset('csv', data_files={'train': train_path, 'validation': val_path})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-Medium")
model = AutoModelForMaskedLM.from_pretrained("snunlp/KR-Medium").to(device)

In [14]:
def tokenization(example):
    return tokenizer(example["title"])

tokenized_dataset = dataset.map(tokenization, batched=True, remove_columns=["guid", "title", "label"])
tokenized_dataset

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 280
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 70
    })
})

In [15]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
aug_loader = DataLoader(tokenized_dataset['train'], batch_size=16, shuffle=False, collate_fn=data_collator)
aug_loader

In [78]:
training_args = TrainingArguments(
    output_dir='./checkpoints/mlm_aug_train',
    num_train_epochs=20,
    warmup_steps=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    seed=42,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    report_to='wandb',
    run_name='mlm_aug_train',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss'
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator
)

In [79]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.702700,2.991549
2,3.507300,3.102431
3,2.995800,2.851748
4,2.857200,2.868818
5,2.776100,2.603578
6,2.551700,2.810954
7,2.401400,3.078926
8,2.307900,2.698636
9,2.282300,3.095192
10,2.088700,2.497844


TrainOutput(global_step=360, training_loss=2.1415236632029218, metrics={'train_runtime': 305.8347, 'train_samples_per_second': 18.311, 'train_steps_per_second': 1.177, 'total_flos': 59180913512448.0, 'train_loss': 2.1415236632029218, 'epoch': 20.0})

In [81]:
wandb.finish()

eval/loss,▃▄▃▃▁▂▄▂▄▁▃▄▄▅▅▄█▃▃▇
eval/runtime,▁▁▅▇▃▃▂▃▃▃▅▃█▃▃▃▁▆▃▃
eval/samples_per_second,█▇▂▁▄▅▅▄▄▄▂▄▁▄▄▄█▂▄▄
eval/steps_per_second,█▇▂▁▄▅▅▄▄▄▂▄▁▄▄▄█▂▄▄
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,▂▄▅▆███▇▇▆▆▅▅▄▃▃▂▂▂▁
train/loss,█▇▆▆▅▅▄▄▄▄▃▂▂▂▂▃▁▁▂▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [16]:
model = AutoModelForMaskedLM.from_pretrained("./checkpoints/mlm_aug_train/checkpoint-180").to("cuda")

In [17]:
def augment_epoch():
    augmented_texts = []

    for batch in tqdm(aug_loader):
        batch = batch.to(device)
        token_logits = model(**batch).logits

        # Loop over each example in batch
        for i in range(batch.input_ids.shape[0]):
            mask_token_index = torch.where(batch.input_ids[i] == tokenizer.mask_token_id)[0]
            mask_token_logits = token_logits[i, mask_token_index, :]
            replaced_tokens = mask_token_logits.argmax(dim=1).tolist()

            # Replace mask tokens by replaced tokens
            augmented_input = batch.input_ids[i].clone()
            for k in range(len(mask_token_index)):
                augmented_input[mask_token_index[k]] = replaced_tokens[k]

            augmented_text = tokenizer.decode(augmented_input, skip_special_tokens=True)
            augmented_texts.append(augmented_text)

    return augmented_texts

In [18]:
train_df = pd.read_csv("data/train_low.csv")

In [39]:
# augmented_data = {
#     "id": train_df["id"].tolist(),
#     "text": train_df["text"].tolist(),
#     "label": train_df["label"].tolist(),
#     }
augmented_data = {"guid": [], "title": [], "label": []}
num_augs = 2

# To avoid duplicates, augment 3 times more than num_augs
for i in tqdm(range(num_augs * 3)):
    augmented_texts = augment_epoch()
    for j, row in train_df.iterrows():
        augmented_data["guid"].append(row["guid"])
        augmented_data["title"].append(augmented_texts[j])
        augmented_data["label"].append(row["label"])

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [40]:
augment_df = pd.DataFrame(augmented_data)
augment_df.sort_values(by=["guid"], inplace=True, kind="mergesort")
augment_df

,guid,title,label
90,ynat-v1_train_00401,홍남기 부총리 출입기자단과 간담회,6
370,ynat-v1_train_00401,홍남기 부총리 출입기자단과 간담회,6
650,ynat-v1_train_00401,홍남기 부총리 출입기자단과 간담회,6
930,ynat-v1_train_00401,홍남기 부총리 출입기자단과 간담회,6
1210,ynat-v1_train_00401,홍남기 부총리 출입기자단과 간담회,6
...,...,...,...
500,ynat-v1_train_45546,사랑의 김장 들어보이는 이해찬,6
780,ynat-v1_train_45546,사랑의 김장 들어보는 이해찬,6
1060,ynat-v1_train_45546,사랑의 김장 들어보이는 이해찬,6
1340,ynat-v1_train_45546,사랑의 김장 돋보이는 이해찬,6


In [41]:
augment_df.drop_duplicates(subset=["title"], inplace=True)
augment_df = augment_df[~augment_df["title"].isin(train_df["title"].tolist())]
augment_df.drop(augment_df[augment_df["title"] == ""].index, inplace=True)
augment_df.sort_values(by=["guid"], inplace=True, kind="mergesort")
augment_df["guid"].value_counts()

<ipython-input-41-7728a1965a3e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  augment_df.drop(augment_df[augment_df["title"] == ""].index, inplace=True)
<ipython-input-41-7728a1965a3e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  augment_df.sort_values(by=["guid"], inplace=True, kind="mergesort")


ynat-v1_train_44846    6
ynat-v1_train_22681    6
ynat-v1_train_14649    6
ynat-v1_train_41985    6
ynat-v1_train_22019    6
                      ..
ynat-v1_train_24497    1
ynat-v1_train_10025    1
ynat-v1_train_24529    1
ynat-v1_train_36790    1
ynat-v1_train_30173    1
Name: guid, Length: 266, dtype: int64

In [42]:
augment_df

,guid,title,label
102,ynat-v1_train_00492,새누리당 예비후보 의정보고회 금지 가처분 신청,6
382,ynat-v1_train_00492,새누리당 예비후보 의정보고회 금지가처분 신청,6
662,ynat-v1_train_00492,대구 예비후보 의정보고회 금지 가결종합,6
942,ynat-v1_train_00492,대구 예비후보 의경고회 금지 가처분 신청,6
1502,ynat-v1_train_00492,대구 예비얗 의정보고회 금지 가처분 신청,6
...,...,...,...
659,ynat-v1_train_45312,예루살렘 팔레스 민간인 주거지 방문 이스라엘에 비난 쇄도,4
939,ynat-v1_train_45312,##촘루살렘 팔레스타인 주거지 철거 이스라엘에 비난 쇄도,4
1219,ynat-v1_train_45312,예루살렘 팔레스타인 주거지서 이스라엘에 비난 쇄도,4
780,ynat-v1_train_45546,사랑의 김장 들어보는 이해찬,6


In [23]:
# Drop empty texts
len(augment_df)

4205

In [29]:
final_df = augment_df.groupby("guid").sample(n=2)
final_df

,guid,title,label
6810,ynat-v1_train_00401,홍남기 부총리 출입기자단과 기자 토론회,6
1210,ynat-v1_train_00401,채워남기 부총리 출입기자단과 간담회,6
102,ynat-v1_train_00492,대구광역시회 의정보고회 금지 가처분 신청,6
4862,ynat-v1_train_00492,대구 예비후보자광고회 금지 가처분 신청,6
9713,ynat-v1_train_00804,박상기 법무부 장관료가 인사청문회 13일 실시,6
...,...,...,...
384,ynat-v1_train_45276,풀무원 자 햊주 매각 추진 보도 사실 무근,1
1219,ynat-v1_train_45312,예루살렘 팔레스타인 주거지 철거 이스라엘에 항의 쇄도,4
3459,ynat-v1_train_45312,예루살렘 팔레스타인 주거지 인근 이스라엘에 비난 쇄도,4
5820,ynat-v1_train_45546,사랑의 김장 들어라 이해찬,6


In [31]:
train_df

,guid,title,label
0,ynat-v1_train_20804,군 복무 후 용병급 활약 두산 정수빈 결승타까지,5
1,ynat-v1_train_40850,하나금융투자 IT서비스 국제인증 획득,0
2,ynat-v1_train_15259,방송대 청각장애학생 위한 매체강의 자막 서비스 제공,2
3,ynat-v1_train_37602,니카라과 망명 중인 엘살바도르 전 대통령에 시민권,4
4,ynat-v1_train_23775,신간 신우인 목사의 이스라엘 왕 이야기,3
...,...,...,...
275,ynat-v1_train_12293,독일 프리랜서 기자가 만든 세월호 다큐 베를린 등지서 상영,4
276,ynat-v1_train_20900,질의 듣는 한인섭 한국형사정책연구원 원장,2
277,ynat-v1_train_22610,이란군 최신구축함 등 해군함대 3월부터 5개월간 대서양 항해,4
278,ynat-v1_train_10964,법원행정처장 김학의 재정신청 기각 다시 보는 건 부적절,2


In [34]:
final_df

,guid,title,label
6810,ynat-v1_train_00401,홍남기 부총리 출입기자단과 기자 토론회,6
1210,ynat-v1_train_00401,채워남기 부총리 출입기자단과 간담회,6
102,ynat-v1_train_00492,대구광역시회 의정보고회 금지 가처분 신청,6
4862,ynat-v1_train_00492,대구 예비후보자광고회 금지 가처분 신청,6
9713,ynat-v1_train_00804,박상기 법무부 장관료가 인사청문회 13일 실시,6
...,...,...,...
384,ynat-v1_train_45276,풀무원 자 햊주 매각 추진 보도 사실 무근,1
1219,ynat-v1_train_45312,예루살렘 팔레스타인 주거지 철거 이스라엘에 항의 쇄도,4
3459,ynat-v1_train_45312,예루살렘 팔레스타인 주거지 인근 이스라엘에 비난 쇄도,4
5820,ynat-v1_train_45546,사랑의 김장 들어라 이해찬,6


In [37]:
final_df = pd.concat([train_df, final_df], ignore_index=True)
final_df.sort_values(by=["guid"], inplace=True, kind="mergesort")
final_df

,guid,title,label
90,ynat-v1_train_00401,홍남기 부총리 출입기자단과 간담회,6
280,ynat-v1_train_00401,홍남기 부총리 출입기자단과 기자 토론회,6
281,ynat-v1_train_00401,채워남기 부총리 출입기자단과 간담회,6
102,ynat-v1_train_00492,대구 예비후보 의정보고회 금지 가처분 신청,6
282,ynat-v1_train_00492,대구광역시회 의정보고회 금지 가처분 신청,6
...,...,...,...
836,ynat-v1_train_45312,예루살렘 팔레스타인 주거지 철거 이스라엘에 항의 쇄도,4
837,ynat-v1_train_45312,예루살렘 팔레스타인 주거지 인근 이스라엘에 비난 쇄도,4
220,ynat-v1_train_45546,사랑의 김장 들어보이는 이해찬,6
838,ynat-v1_train_45546,사랑의 김장 들어라 이해찬,6


In [38]:
final_df.to_csv("data/train_mlm_2.csv", index=False)